Connecting to IBKR on port 7497
Error 0: 2104
Error 0: 2104
Error 0: 2104
Error 0: 2106
Error 0: 2106
Error 0: 2106
Error 0: 2158

Found 3 vertical spreads:
                  Position Title        Low Strike Leg  Low Strike Position  Low Strike Market Value       High Strike Leg  High Strike Position  High Strike Market Value  Total Market Value Spread Type Expiry Date
GOOGL Jun 27'25 145/155 Bear Put GOOGL Jun 27'25 145 P                    1                   4680.8 GOOGL Jun 27'25 155 P                    -1                 -10718.93            -6038.13    Bear Put  2025-06-27
    WMT Jun 27'25 84/94 Bear Put    WMT Jun 27'25 84 P                    1                   5766.8    WMT Jun 27'25 94 P                    -1                  -6732.93             -966.13    Bear Put  2025-06-27
 TSLA Jun 27'25 275/285 Bear Put  TSLA Jun 27'25 275 P                    1                  52063.8  TSLA Jun 27'25 285 P                    -1                 -74435.93           -22372.13    Bear

In [2]:
import pandas as pd
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import threading
import time
import queue
from datetime import datetime
from collections import defaultdict

# Configuration
PAPER_TRADING_PORT = 7497
LIVE_TRADING_PORT = 7496
USE_LIVE_TRADING = False  # Set to True for live trading

class PositionWrapper(EWrapper):
    """Wrapper class for IB API callbacks"""
    def __init__(self):
        EWrapper.__init__(self)
        self.positions = []
        self.positions_received = False
        self.data_queue = queue.Queue()
        self.next_req_id = 1000  # Start higher to avoid conflicts
        self.market_data = {}
        self.req_id_to_position = {}  # Maps request IDs to position indices
        self.market_data_received = set()
        
    def error(self, reqId, errorCode, errorString, advancedOrderRejectJson="", *args):
        print(f"Error {reqId}: {errorCode} - {errorString}")
        # Mark failed requests
        if reqId in self.req_id_to_position and errorCode in [321, 200, 354, 10090]:
            self.market_data_received.add(reqId)
        self.data_queue.put(("error", (reqId, errorCode, errorString)))
        
    def position(self, account, contract, position, avgCost):
        if contract.secType == 'OPT' and position != 0:
            self.positions.append({
                'symbol': contract.symbol,
                'expiry': contract.lastTradeDateOrContractMonth,
                'strike': contract.strike,
                'right': contract.right,
                'position': position,
                'avgCost': avgCost,
                'contract': contract
            })
        
    def positionEnd(self):
        self.positions_received = True
        self.data_queue.put(("positions_complete", None))
    
    def tickPrice(self, reqId, tickType, price, attrib):
        """Handle market data price updates"""
        if reqId not in self.market_data:
            self.market_data[reqId] = {}
        
        # Store price based on tick type
        # 1 = bid, 2 = ask, 4 = last, 9 = close
        self.market_data[reqId][tickType] = price
        
        # Mark this request as having received data
        self.market_data_received.add(reqId)
        self.data_queue.put(("tick_price", (reqId, tickType, price)))
    
    def tickSize(self, reqId, tickType, size):
        """Handle market data size updates"""
        pass
    
    def getNextRequestId(self):
        """Get the next available request ID"""
        req_id = self.next_req_id
        self.next_req_id += 1
        return req_id

class PositionClient(EClient):
    """Client class to connect to IB API"""
    def __init__(self, wrapper):
        EClient.__init__(self, wrapper)

class PositionApp:
    """Main application class for getting positions and market data"""
    def __init__(self, host='127.0.0.1', port=7497):
        self.host = host
        self.port = port
        self.wrapper = PositionWrapper()
        self.client = PositionClient(self.wrapper)
        self.connected = False
        self.api_thread = None
    
    def _run_client(self):
        """Run the client in a separate thread"""
        try:
            self.client.run()
        except Exception as e:
            print(f"Error in client thread: {str(e)}")
    
    def connect(self):
        """Connect to IBKR"""
        print(f"Connecting to IBKR at {self.host}:{self.port}")
        
        try:
            self.client.connect(self.host, self.port, clientId=2)  # Use different client ID
        except Exception as e:
            print(f"Connection error: {str(e)}")
            return False
        
        # Start API thread
        self.api_thread = threading.Thread(target=self._run_client, daemon=True)
        self.api_thread.start()
        
        # Wait for connection
        time.sleep(2)
        
        # Set market data type to delayed
        self.client.reqMarketDataType(3)  # 3 = delayed data
        
        self.connected = True
        return True
    
    def disconnect(self):
        """Disconnect from IBKR"""
        if self.connected:
            print("Disconnecting from IBKR")
            try:
                self.client.disconnect()
                self.connected = False
            except Exception as e:
                print(f"Error disconnecting: {str(e)}")
    
    def create_option_contract(self, symbol, expiry, strike, right):
        """Create an option contract"""
        contract = Contract()
        contract.symbol = symbol
        contract.secType = "OPT"
        contract.exchange = "SMART"
        contract.currency = "USD"
        contract.lastTradeDateOrContractMonth = expiry
        contract.strike = strike
        contract.right = right
        return contract
    
    def get_current_price(self, market_data):
        """Get the best available current price from market data"""
        if not market_data:
            return None
        
        # Priority: 1) Last price, 2) Mid-point of bid/ask, 3) Close price, 4) Individual bid/ask
        last = market_data.get(4)  # Last price
        bid = market_data.get(1)   # Bid
        ask = market_data.get(2)   # Ask
        close = market_data.get(9) # Close
        
        # If we have last price and it's valid
        if last is not None and last > 0:
            return last
        
        # If we have both bid and ask, use mid-point
        if bid is not None and ask is not None and bid > 0 and ask > 0:
            return (bid + ask) / 2
        
        # Fall back to close price
        if close is not None and close > 0:
            return close
        
        # If we only have bid or ask, use that
        if ask is not None and ask > 0:
            return ask
        if bid is not None and bid > 0:
            return bid
        
        return None
    
    def get_positions_with_market_data(self):
        """Get positions and fetch current market data"""
        if not self.connected:
            return []
        
        # Request positions
        self.client.reqPositions()
        
        # Wait for positions
        timeout = 10
        start = time.time()
        while not self.wrapper.positions_received and (time.time() - start) < timeout:
            try:
                msg_type, msg_data = self.wrapper.data_queue.get(timeout=0.1)
                if msg_type == "positions_complete":
                    break
            except queue.Empty:
                continue
        
        if not self.wrapper.positions_received:
            print("Timeout waiting for positions")
            return []
        
        print(f"Found {len(self.wrapper.positions)} option positions")
        
        # Request market data for each position
        print("Requesting market data for positions...")
        
        for i, pos in enumerate(self.wrapper.positions):
            req_id = self.wrapper.getNextRequestId()
            self.wrapper.req_id_to_position[req_id] = i
            
            # Create contract for market data request
            contract = self.create_option_contract(
                pos['symbol'], pos['expiry'], pos['strike'], pos['right']
            )
            
            print(f"Requesting data for {pos['symbol']} {pos['strike']} {pos['right']} (reqId: {req_id})")
            
            try:
                # Request market data snapshot
                self.client.reqMktData(req_id, contract, "", True, False, [])
                time.sleep(0.2)  # Small delay between requests
            except Exception as e:
                print(f"Error requesting market data for reqId {req_id}: {e}")
                self.wrapper.market_data_received.add(req_id)
        
        # Wait for market data
        print("Waiting for market data...")
        timeout = 15
        start = time.time()
        expected_requests = len(self.wrapper.req_id_to_position)
        
        while len(self.wrapper.market_data_received) < expected_requests and (time.time() - start) < timeout:
            try:
                msg_type, msg_data = self.wrapper.data_queue.get(timeout=0.5)
                if msg_type == "tick_price":
                    req_id, tick_type, price = msg_data
                    pos_idx = self.wrapper.req_id_to_position.get(req_id)
                    if pos_idx is not None:
                        pos = self.wrapper.positions[pos_idx]
                        print(f"Received price for {pos['symbol']} {pos['strike']}: ${price} (type: {tick_type})")
            except queue.Empty:
                continue
        
        print(f"Received market data for {len(self.wrapper.market_data_received)}/{expected_requests} positions")
        
        # Cancel all market data requests
        for req_id in self.wrapper.req_id_to_position.keys():
            try:
                self.client.cancelMktData(req_id)
            except:
                pass
        
        # Add market data to positions
        for req_id, pos_idx in self.wrapper.req_id_to_position.items():
            if pos_idx < len(self.wrapper.positions):
                market_data = self.wrapper.market_data.get(req_id, {})
                self.wrapper.positions[pos_idx]['market_data'] = market_data
                
                # Debug: Show market data received
                if market_data:
                    pos = self.wrapper.positions[pos_idx]
                    print(f"Market data for {pos['symbol']} {pos['strike']}: {market_data}")
        
        return self.wrapper.positions

def get_vertical_spreads():
    # Connect to IBKR
    port = LIVE_TRADING_PORT if USE_LIVE_TRADING else PAPER_TRADING_PORT
    app = PositionApp(port=port)
    
    if not app.connect():
        print("Failed to connect to IBKR")
        return pd.DataFrame()
    
    try:
        # Get positions with market data
        positions = app.get_positions_with_market_data()
        
        if not positions:
            print("No option positions found")
            return pd.DataFrame()
        
        # Process positions into spreads
        spreads = []
        grouped = defaultdict(list)
        
        # Group by symbol, expiry, and option type
        for pos in positions:
            key = (pos['symbol'], pos['expiry'], pos['right'])
            grouped[key].append(pos)
        
        # Find vertical spreads
        for (symbol, expiry, right), position_group in grouped.items():
            if len(position_group) >= 2:
                position_group.sort(key=lambda x: x['strike'])
                
                for i in range(len(position_group)):
                    for j in range(i+1, len(position_group)):
                        pos1, pos2 = position_group[i], position_group[j]
                        
                        # Check for opposite positions (vertical spread)
                        if (pos1['position'] > 0) != (pos2['position'] > 0):
                            spread_type = get_spread_type(pos1, pos2, right)
                            if spread_type:
                                spreads.append(create_spread_row(pos1, pos2, spread_type, app))
        
        return pd.DataFrame(spreads)
    
    finally:
        app.disconnect()

def get_spread_type(pos1, pos2, right):
    """Determine spread type - pos1 is lower strike"""
    if right == 'P':  # Puts
        if pos1['position'] < 0 and pos2['position'] > 0:
            return 'Bull Put'
        elif pos1['position'] > 0 and pos2['position'] < 0:
            return 'Bear Put'
    elif right == 'C':  # Calls
        if pos1['position'] > 0 and pos2['position'] < 0:
            return 'Bull Call'
        elif pos1['position'] < 0 and pos2['position'] > 0:
            return 'Bear Call'
    return None

def create_spread_row(pos1, pos2, spread_type, app):
    """Create DataFrame row for the spread with current market values and opening costs"""
    symbol = pos1['symbol']
    expiry = format_date(pos1['expiry'])
    low_strike = int(min(pos1['strike'], pos2['strike']))
    high_strike = int(max(pos1['strike'], pos2['strike']))
    
    # Determine which position is which strike
    if pos1['strike'] < pos2['strike']:
        low_pos, high_pos = pos1, pos2
    else:
        low_pos, high_pos = pos2, pos1
    
    title = f"{symbol} {expiry} {low_strike}/{high_strike} {spread_type}"
    low_leg = f"{symbol} {expiry} {low_strike} {pos1['right']}"
    high_leg = f"{symbol} {expiry} {high_strike} {pos1['right']}"
    
    # Calculate opening costs (avgCost is already the full contract value)
    low_strike_opening_cost = float(low_pos['avgCost']) * float(low_pos['position'])
    high_strike_opening_cost = float(high_pos['avgCost']) * float(high_pos['position'])
    total_opening_cost = low_strike_opening_cost + high_strike_opening_cost
    
    # Get current market prices
    low_current_price = app.get_current_price(low_pos.get('market_data', {}))
    high_current_price = app.get_current_price(high_pos.get('market_data', {}))
    
    # Calculate current market values
    if low_current_price is not None:
        low_strike_market_value = low_current_price * float(low_pos['position'])*100
    else:
        low_strike_market_value = low_strike_opening_cost  # Fall back to opening cost
        
    if high_current_price is not None:
        high_strike_market_value = high_current_price * float(high_pos['position'])*100
    else:
        high_strike_market_value = high_strike_opening_cost  # Fall back to opening cost
    
    total_market_value = low_strike_market_value + high_strike_market_value
    
    return {
        'Position Title': title,
        'Low Strike Leg': low_leg,
        'Low Strike Position': int(low_pos['position']),
        'Low Strike Opening Cost': round(low_strike_opening_cost, 2),
        'Low Strike Market Value': round(low_strike_market_value, 2),
        'High Strike Leg': high_leg,
        'High Strike Position': int(high_pos['position']),
        'High Strike Opening Cost': round(high_strike_opening_cost, 2),
        'High Strike Market Value': round(high_strike_market_value, 2),
        'Total Opening Cost': round(total_opening_cost, 2),
        'Total Market Value': round(total_market_value, 2),
        'P&L': round(total_market_value - total_opening_cost, 2),
        'Spread Type': spread_type,
        'Expiry Date': datetime.strptime(pos1['expiry'], '%Y%m%d').date()
    }

def format_date(expiry_str):
    """Convert YYYYMMDD to 'Mon DD'YY' format"""
    dt = datetime.strptime(expiry_str, '%Y%m%d')
    return dt.strftime("%b %d'%y")

if __name__ == "__main__":
    print(f"Connecting to IBKR on port {LIVE_TRADING_PORT if USE_LIVE_TRADING else PAPER_TRADING_PORT}")
    
    print("\nNote: Using the same market data approach as your working price_monitor system")
    print("- Delayed market data (type 3)")
    print("- Proper threading and queue handling") 
    print("- Fallback to opening costs if market data unavailable")
    
    df = get_vertical_spreads()
    
    if not df.empty:
        print(f"\nFound {len(df)} vertical spreads:")
        print(df.to_string(index=False))
        
        # Optional: Display with better formatting
        print("\n" + "="*100)
        print("DETAILED BREAKDOWN:")
        print("="*100)
        
        for _, row in df.iterrows():
            print(f"\n{row['Position Title']}")
            print(f"  Low Strike:  {row['Low Strike Leg']} | Position: {row['Low Strike Position']}")
            print(f"    Opening Cost: ${row['Low Strike Opening Cost']:,.2f} | Current Market Value: ${row['Low Strike Market Value']:,.2f}")
            print(f"  High Strike: {row['High Strike Leg']} | Position: {row['High Strike Position']}")
            print(f"    Opening Cost: ${row['High Strike Opening Cost']:,.2f} | Current Market Value: ${row['High Strike Market Value']:,.2f}")
            print(f"  TOTALS - Opening Cost: ${row['Total Opening Cost']:,.2f} | Market Value: ${row['Total Market Value']:,.2f} | P&L: ${row['P&L']:,.2f}")
            print(f"  Expiry: {row['Expiry Date']}")
    else:
        print("No vertical spreads found")

Connecting to IBKR on port 7497

Note: Using the same market data approach as your working price_monitor system
- Delayed market data (type 3)
- Proper threading and queue handling
- Fallback to opening costs if market data unavailable
Connecting to IBKR at 127.0.0.1:7497
Error -1: 0 - 2104
Error -1: 0 - 2104
Error -1: 0 - 2104
Error -1: 0 - 2104
Error -1: 0 - 2104
Error -1: 0 - 2104
Error -1: 0 - 2104
Error -1: 0 - 2104
Error -1: 0 - 2104
Error -1: 0 - 2106
Error -1: 0 - 2106
Error -1: 0 - 2106
Error -1: 0 - 2158
Found 6 option positions
Requesting market data for positions...
Requesting data for GOOGL 145.0 P (reqId: 1000)
Requesting data for WMT 84.0 P (reqId: 1001)
Requesting data for GOOGL 155.0 P (reqId: 1002)
Requesting data for TSLA 275.0 P (reqId: 1003)
Requesting data for WMT 94.0 P (reqId: 1004)
Requesting data for TSLA 285.0 P (reqId: 1005)
Waiting for market data...
Received market data for 6/6 positions
Market data for GOOGL 145.0: {1: -1.0, 2: -1.0, 9: 0.14}
Market data 

In [3]:
df

,Position Title,Low Strike Leg,Low Strike Position,Low Strike Opening Cost,Low Strike Market Value,High Strike Leg,High Strike Position,High Strike Opening Cost,High Strike Market Value,Total Opening Cost,Total Market Value,P&L,Spread Type,Expiry Date
0,GOOGL Jun 27'25 145/155 Bear Put,GOOGL Jun 27'25 145 P,1,46.81,14.0,GOOGL Jun 27'25 155 P,-1,-107.19,-26.0,-60.38,-12.0,48.38,Bear Put,2025-06-27
1,WMT Jun 27'25 84/94 Bear Put,WMT Jun 27'25 84 P,1,57.67,13.0,WMT Jun 27'25 94 P,-1,-67.33,-120.0,-9.66,-107.0,-97.34,Bear Put,2025-06-27
2,TSLA Jun 27'25 275/285 Bear Put,TSLA Jun 27'25 275 P,1,520.64,255.0,TSLA Jun 27'25 285 P,-1,-744.36,-392.0,-223.72,-137.0,86.72,Bear Put,2025-06-27
